<a href="https://colab.research.google.com/github/hj10330149/Leetcode-Practice/blob/main/IMBD%E5%88%9D%E8%B3%BDcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 資料處理

下載 auto-sklearn & lightgbm 套件

In [ ]:
pip install auto-sklearn

In [ ]:
pip install lightgbm

載入Pandas, Numpy, autosklearn, lightgbm 以及 sklearn 的 metrics, KMeans, GridSearchCV 套件

In [ ]:
import pandas as pd
import numpy as np
import autosklearn.regression
import lightgbm as lgbm
import sklearn.metrics
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV

讀取訓練資料 df_train、 測試資料 df_test

In [ ]:
df_train=pd.read_excel('2022-train-v2.xlsx')
df_test=pd.read_excel('2022-test-v1.xlsx')

以0取代資料中的遺失值NA，另外將訓練資料中的 feature (製程中感測器量測的數據) 跟 output (工件噴漆的膜厚) 分出來

In [ ]:
# 遺失值補0
df_train=df_train.fillna(0)
df_test=df_test.fillna(0)
# 將訓練資料中的 feature 和 output 切分出來
x = df_train.iloc[:,6:132]
y = df_train.iloc[:,0:6]
# 對feature做標準化
mean = x.mean()
sd = x.std()
train_x_scaled = (x - mean)/sd
test_x_scaled = (df_test - mean)/sd
train_x_scaled.reset_index(inplace = True)
test_x_scaled.reset_index(inplace = True)

# 建立 autoML 模型

In [ ]:
def automl(x_scaled,y,clusters,random_state):
  # 分群
  globals()['kmeans_'+str(clusters)]=KMeans(n_clusters=clusters,random_state=random_state).fit(x_scaled.iloc[:,1:])
  label=pd.DataFrame(globals()['kmeans_'+str(clusters)].labels_)
  label.columns=['label']
  x_scaled=pd.concat([x_scaled,label],axis=1)
  data=pd.concat([y,x_scaled],axis=1) # 有Y有index有X有label

  # 各群建模
  for c in range(1,clusters+1): # 先拆成各群
    globals()['group_'+str(c)]=data.loc[data['label']==c-1]
    globals()['group_'+str(c)]=(globals()['group_'+str(c)]).drop(['label'],axis=1) # 各群Y和X(包含index)
    X_train=globals()['group_'+str(c)].iloc[:,7:] # 不包含index
    Y_train=globals()['group_'+str(c)].iloc[:,0:6]
    for i in range(0,6): # 再拆各個Y，並對每個Y建模
      globals()['automl_Y'+str(i+1)+'_Cluster_'+str(c)]=autosklearn.regression.AutoSklearnRegressor(time_left_for_this_task=120,per_run_time_limit=30) #建模
      globals()['automl_Y'+str(i+1)+'_Cluster_'+str(c)].fit(X_train,Y_train.iloc[:,i]) # 用X_train, Y_train去訓練模型，調整模型參數
      globals()['train_predictions_Y'+str(i+1)+'_Cluster_'+str(c)]=globals()['automl_Y'+str(i+1)+'_Cluster_'+str(c)].predict(X_train) # 用X_train資料去預測Y

  # 合併各群的X
  x=pd.DataFrame()
  for c in range(1,clusters+1):
    x=pd.concat([x,globals()['group_'+str(c)].iloc[:,6:]],axis=0) # 合併所有群的X(包含index)

  # 合併各群的預測值Y
  df_train_predict=pd.DataFrame()
  for i in range(0,6):
    globals()['(Train_prediction)AutoML_Y'+str(i+1)]=pd.DataFrame()
    for c in range(1,clusters+1):
      globals()['(Train_prediction)AutoML_Y'+str(i+1)]=pd.concat([globals()['(Train_prediction)AutoML_Y'+str(i+1)], pd.DataFrame(globals()['train_predictions_Y'+str(i+1)+'_Cluster_'+str(c)])],axis=0)
    df_train_predict = pd.concat([df_train_predict, globals()['(Train_prediction)AutoML_Y'+str(i+1)]], axis=1)
  df_train_predict.columns = ['Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'Y6']

  # 合併預測值Y與X
  df_train_predict.reset_index(drop=True, inplace = True)
  x.reset_index(drop=True, inplace = True)
  result = pd.concat([df_train_predict, x], axis=1)
  train_prediction_result = result.sort_values(by=['index']).iloc[:, :6]
  return train_prediction_result

In [ ]:
train_prediction_result_automl_c1=automl(train_x_scaled,y,1,123)

In [ ]:
def auto_test(x_scaled,clusters):
  # 分群
  label=pd.DataFrame(globals()['kmeans_'+str(clusters)].predict(x_scaled.iloc[:,1:]))
  label.columns=['label']
  data=pd.concat([x_scaled,label],axis=1)

  # 各群預測
  for c in range(1,clusters+1): # 先拆成k群
    globals()['(test)group_'+str(c)]=data.loc[data['label']==c-1]
    globals()['(test)group_'+str(c)]=(globals()['(test)group_'+str(c)]).drop(['label'],axis=1) # 各群的X(包含index)
    X_test=globals()['(test)group_'+str(c)].iloc[:,1:] # 不包含index
    for i in range(0,6): # 再拆各個Y，並對每個Y預測
      globals()['test_predictions_Y'+str(i+1)+'_Cluster_'+str(c)]=globals()['automl_Y'+str(i+1)+'_Cluster_'+str(c)].predict(X_test) # 各群各個Y的fit值

  # 合併各群的X
  x=pd.DataFrame()
  for c in range(1,clusters+1):
    x=pd.concat([x,globals()['(test)group_'+str(c)]],axis=0) # 合併所有群的X(包含index)

  # 合併各群的預測值Y
  df_test_predict=pd.DataFrame()
  for i in range(0,6):
    globals()['(Test_prediction)AutoML_Y'+str(i+1)]=pd.DataFrame()
    for c in range(1,clusters+1):
      globals()['(Test_prediction)AutoML_Y'+str(i+1)]=pd.concat([globals()['(Test_prediction)AutoML_Y'+str(i+1)], pd.DataFrame(globals()['test_predictions_Y'+str(i+1)+'_Cluster_'+str(c)])],axis=0)
    df_test_predict = pd.concat([df_test_predict, globals()['(Test_prediction)AutoML_Y'+str(i+1)]], axis=1)
  df_test_predict.columns = ['Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'Y6']

  # 合併預測值Y與X
  df_test_predict.reset_index(drop=True, inplace = True)
  x.reset_index(drop=True, inplace = True)
  result = pd.concat([df_test_predict, x], axis=1)
  test_prediction_result = result.sort_values(by=['index']).iloc[:, :6]
  return test_prediction_result


In [ ]:
test_prediction_result_automl_c1=auto_test(test_x_scaled,1)

# 建立 LightGBM 模型

In [ ]:
def lgbm_model(x_scaled, Y, clusters, random_state):
  final_model = {}

  # 分群
  globals()['kmeans_'+str(clusters)]=KMeans(n_clusters=clusters,random_state=random_state).fit(x_scaled.iloc[:, 1:])
  label=pd.DataFrame(globals()['kmeans_'+str(clusters)].labels_)
  label.columns=['label']
  x_scaled=pd.concat([x_scaled,label],axis=1) # axis=1 對行合併
  data=pd.concat([Y, x_scaled],axis=1)

  # 利用分群結果將資料分成數群
  for c in range(1,clusters+1):
    locals()['group_'+str(c)]=data.loc[data['label']==c-1]
    locals()['group_'+str(c)]=(locals()['group_'+str(c)]).drop(['label'],axis=1)

  # 建模
  for c in range(1,clusters+1):
    x = locals()['group_'+str(c)].iloc[:,7:]
    Y = locals()['group_'+str(c)].iloc[:,:6]
    for i in range(0,6):
      locals()['Y'+str(i+1)]=Y.iloc[:,i]
      other_params = {
          'boosting_type': 'gbdt',
          'objective': 'regression',
          'metric': 'rmse',
          'n_estimators': 10,
          'min_child_sample': 5,
          'max_depth': 3,
          'num_leaves': 5,
          'reg_lambda': 0.1,
          'learning_rate': 0.01
      }
      model = lgbm.LGBMRegressor(**other_params)
      cv_params = {
          'n_estimators': [10, 20, 30],
          'min_child_sample': [3, 4, 5],
          'max_depth': [2, 3],
          'num_leaves': [3, 4, 5],
          'reg_lambda': [0.02, 0.05, 0.1],
          'learning_rate': [0.01, 0.03, 0.05]
          }
      optimized_model = GridSearchCV(
          estimator=model,
          param_grid=cv_params,
          scoring='neg_mean_squared_error',
          cv=3,
          verbose=0)
      optimized_model.fit(x,locals()['Y'+str(i+1)])
      for key in optimized_model.best_params_:
        other_params[key] = optimized_model.best_params_[key]

      optimized_model = lgbm.LGBMRegressor(**other_params)
      optimized_model.fit(x,locals()['Y'+str(i+1)])

      locals()['LGBM_Y'+str(i+1)+'_Cluster_'+str(c)] = pd.DataFrame(optimized_model.predict(x))
      final_model['Y'+str(i+1)+'_Cluster_'+str(c)] = optimized_model

  # 整理預測結果
  x = pd.DataFrame()
  Y_prediction = pd.DataFrame()
  for i in range(6):
    locals()['LGBM_predictions_Y'+str(i+1)] = pd.DataFrame()
    for c in range(1, clusters+1):
      locals()['LGBM_predictions_Y'+str(i+1)] = pd.concat([locals()['LGBM_predictions_Y'+str(i+1)], locals()['LGBM_Y'+str(i+1)+'_Cluster_'+str(c)]],axis=0)
    Y_prediction = pd.concat([Y_prediction, locals()['LGBM_predictions_Y'+str(i+1)]], axis=1)
  Y_prediction.columns = ['Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'Y6']

  for c in range(1, clusters+1):
    x = pd.concat([x, locals()['group_'+str(c)].iloc[:,6:]], axis=0)

  Y_prediction.reset_index(drop=True, inplace = True)
  x.reset_index(drop=True, inplace = True)
  result = pd.concat([Y_prediction, x], axis=1)
  prediction_result = result.sort_values(by=['index']).iloc[:, :6]

  # 輸出
  return final_model, prediction_result

In [ ]:
final_model_lgbm6, train_prediction_result_lgbm6 = lgbm_model(train_x_scaled, y, 6, 123)

In [ ]:
final_model_lgbm4, train_prediction_result_lgbm4 = lgbm_model(train_x_scaled, y, 4, 123)

In [ ]:
def lgbm_predict(x_scaled, clusters, model):

  # 分群
  label=pd.DataFrame(globals()['kmeans_'+str(clusters)].predict(x_scaled.iloc[:,1:]))
  label.columns=['label']
  data=pd.concat([x_scaled,label],axis=1)

  # 各群建模
  for c in range(1,clusters+1): # 先拆成各群
    globals()['(test)group_'+str(c)]=data.loc[data['label']==c-1]
    globals()['(test)group_'+str(c)]=(globals()['(test)group_'+str(c)]).drop(['label'],axis=1) # 各群的X(包含index)
    X_test=globals()['(test)group_'+str(c)].iloc[:,1:] # 不包含index
    for i in range(0,6): # 在拆各個Y，並對每個Y建模
      globals()['test_predictions_Y'+str(i+1)+'_Cluster_'+str(c)]=pd.DataFrame(model['Y'+str(i+1)+'_Cluster_'+str(c)].predict(X_test)) # 各群各個Y的fit值
  # 合併各群的X
  x=pd.DataFrame()
  for c in range(1,clusters+1):
    x=pd.concat([x,globals()['(test)group_'+str(c)]],axis=0) # 合併所有群的X(包含index)
  # 合併各群的預測值Y
  test_predict=pd.DataFrame()
  for i in range(0,6):
    globals()['(Test_prediction)lgbm_Y'+str(i+1)]=pd.DataFrame()
    for c in range(1,clusters+1):
      globals()['(Test_prediction)lgbm_Y'+str(i+1)]=pd.concat([globals()['(Test_prediction)lgbm_Y'+str(i+1)], globals()['test_predictions_Y'+str(i+1)+'_Cluster_'+str(c)]],axis=0)
    test_predict = pd.concat([test_predict, globals()['(Test_prediction)lgbm_Y'+str(i+1)]], axis=1)
  test_predict.columns = ['Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'Y6']
  # 合併預測值Y與X
  test_predict.reset_index(drop=True, inplace = True)
  x.reset_index(drop=True, inplace = True)
  result = pd.concat([test_predict, x], axis=1)
  test_prediction_result = result.sort_values(by=['index']).iloc[:, :6]
  return test_prediction_result

In [ ]:
test_prediction_result_lgbm4 = lgbm_predict(test_x_scaled, 4, final_model_lgbm4)

In [ ]:
test_prediction_result_lgbm6 = lgbm_predict(test_x_scaled, 6, final_model_lgbm6)

# train資料預測結果合併

In [ ]:
train_Y_prediction = np.array([])

In [ ]:
for i in range(6):
  if i == 0 or i == 4 or i == 5:
    train_Y_prediction = np.append(train_Y_prediction,np.array(train_prediction_result_lgbm6.iloc[:, i]))
  elif i == 1 or i == 3:
    train_Y_prediction = np.append(train_Y_prediction,np.array(train_prediction_result_automl_c1.iloc[:, i]))
  elif i == 2:
    train_Y_prediction = np.append(train_Y_prediction,np.array(train_prediction_result_lgbm4.iloc[:, i]))

In [ ]:
train_Y_prediction=pd.DataFrame(np.transpose(train_Y_prediction.reshape(6,635)))
train_Y_prediction.columns=['Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'Y6']

In [ ]:
# 計算RMSE
sum = 0
for i in range(0,6):
  RMSE = sklearn.metrics.mean_squared_error(train_Y_prediction.iloc[:, i], y.iloc[:, i],squared=False)
  print('Y'+str(i+1)+' RMSE score', RMSE)
  sum=sum+RMSE
print('RMSE score',sum/6)

Y1 RMSE score 8.711065857289856
Y2 RMSE score 11.367351574988438
Y3 RMSE score 19.250008736822885
Y4 RMSE score 14.2315118028477
Y5 RMSE score 13.487139197205128
Y6 RMSE score 14.799731071270601
RMSE score 13.641134706737434


# test資料預測結果合併

In [ ]:
test_Y_prediction = pd.DataFrame()

In [ ]:
for i in range(6):
  if i == 0 or i == 4 or i == 5:
    test_Y_prediction = np.append(test_Y_prediction,np.array(test_prediction_result_lgbm6.iloc[:, i]))
  elif i == 1 or i == 3:
    test_Y_prediction = np.append(test_Y_prediction,np.array(test_prediction_result_automl_c1.iloc[:, i]))
  elif i == 2:
    test_Y_prediction = np.append(test_Y_prediction,np.array(test_prediction_result_lgbm4.iloc[:, i]))

In [ ]:
test_Y_prediction=pd.DataFrame(np.transpose(test_Y_prediction.reshape(6,100)))
test_Y_prediction.columns=['Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'Y6']

In [ ]:
test_Y_prediction

,Y1,Y2,Y3,Y4,Y5,Y6
0,53.206897,69.930804,83.073765,40.148184,69.689655,63.896552
1,53.206897,70.047991,107.540077,36.456063,69.689655,63.896552
2,53.206897,73.757406,107.540077,37.424090,69.689655,63.896552
3,77.198561,55.640552,85.978521,42.033407,62.831507,79.908411
4,77.198561,55.640552,85.978521,42.033407,62.831507,79.908411
...,...,...,...,...,...,...
95,75.193856,64.535163,93.311121,103.381813,84.700699,93.286030
96,75.218846,68.827935,92.020873,94.958838,84.700699,91.639695
97,75.728371,66.760036,93.311121,98.381096,84.700699,91.639695
98,72.953625,60.610259,112.559737,117.350147,97.468984,92.747926


存成CSV檔

In [ ]:
from pathlib import Path
filepath = Path('C:/Users/user/Desktop/Wu-SPC/test_result.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
test_Y_prediction.to_csv(filepath)